# Déploiement d'un modèle de ML dans Azure Kubernetes Service (AKS)
<br>

<img src="https://tse3.mm.bing.net/th?id=OIP.SbDe38X6ihrd_7mG-eeb_wHaC1&pid=Api">


In [39]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import Image
from azureml.core.model import Model

In [40]:
import azureml.core
print(azureml.core.VERSION)

1.0.17


# 1. Connexion workspace Azure ML service
Load existing workspace from the config file info.

In [41]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Found the config file in: C:\Users\seretkow\notebooks\Labs Azure ML service\aml_config\config.json
MLServiceWorkspace
mlserviceresourcegroup
westeurope
70b8f39e-8863-49f7-b6ba-34a80799550c


# 2. Enregistrement du modèle de ML
Register an existing trained model, add descirption and tags.

In [42]:
#Register the model
from azureml.core.model import Model
model = Model.register(model_path = "sklearn_regression_model.pkl", # this points to a local file
                       model_name = "sklearn_regression_model.pkl", # this is the name the model is registered as
                       tags = {'area': "diabetes", 'type': "regression"},
                       description = "Ridge regression model to predict diabetes",
                       workspace = ws)

print(model.name, model.description, model.version)

Registering model sklearn_regression_model.pkl
sklearn_regression_model.pkl Ridge regression model to predict diabetes 5


# 3. Création de l'image
Create an image using the registered model the script that will load and run the model.

In [43]:
%%writefile score.py
import pickle
import json
import numpy
from sklearn.externals import joblib
from sklearn.linear_model import Ridge
from azureml.core.model import Model

def init():
    global model
    # note here "sklearn_regression_model.pkl" is the name of the model registered under
    # this is a different behavior than before when the code is run locally, even though the code is the same.
    model_path = Model.get_model_path('sklearn_regression_model.pkl')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)

# note you can pass in multiple rows for scoring
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = numpy.array(data)
        result = model.predict(data)
        # you can return any data type as long as it is JSON-serializable
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

Overwriting score.py


In [44]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [45]:
%%time
# Prévoir 5 min environ

from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv.yml",
                                                  description = "Image with ridge regression model",
                                                  tags = {'area': "diabetes", 'type': "regression"}
                                                 )

image = ContainerImage.create(name = "myimage1",
                              # this is the model object
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

Creating image
Running......................................................
SucceededImage creation operation finished for image myimage1:5, operation "Succeeded"
Wall time: 4min 51s


# 4. Provisionnement Azure Kubernetes Service (AKS)

This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it.
> https://docs.microsoft.com/en-us/azure/aks/

In [46]:
# Configuration AKS
prov_config = AksCompute.provisioning_configuration()

aks_name = 'my-aks-9' 

aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

In [47]:
%%time
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None
Wall time: 484 ms


## 4.2 Etape optionnelle : attachement à une instance AKS existante

If you have existing AKS cluster in your Azure subscription, you can attach it to the Workspace.

In [48]:
'''
# Use the default configuration (can also provide parameters to customize)
resource_id = '/subscriptions/92c76a2f-0e1c-4216-b65e-abf7a3f34c1e/resourcegroups/raymondsdk0604/providers/Microsoft.ContainerService/managedClusters/my-aks-0605d37425356b7d01'

create_name='my-existing-aks' 
# Create the cluster
attach_config = AksCompute.attach_configuration(resource_id=resource_id)
aks_target = ComputeTarget.attach(workspace=ws, name=create_name, attach_configuration=attach_config)
# Wait for the operation to complete
aks_target.wait_for_completion(True)
'''

"\n# Use the default configuration (can also provide parameters to customize)\nresource_id = '/subscriptions/92c76a2f-0e1c-4216-b65e-abf7a3f34c1e/resourcegroups/raymondsdk0604/providers/Microsoft.ContainerService/managedClusters/my-aks-0605d37425356b7d01'\n\ncreate_name='my-existing-aks' \n# Create the cluster\nattach_config = AksCompute.attach_configuration(resource_id=resource_id)\naks_target = ComputeTarget.attach(workspace=ws, name=create_name, attach_configuration=attach_config)\n# Wait for the operation to complete\naks_target.wait_for_completion(True)\n"

# 5. Déploiement du Web Service dans Azure Kubernetes Service

In [49]:
aks_config = AksWebservice.deploy_configuration()

In [50]:
%%time
# Compter entre 1 à 2 minutes pour déployer
aks_service_name ='aks-service-1'

aks_service = Webservice.deploy_from_image(workspace = ws, 
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Creating service
Running.........
SucceededAKS service creation operation finished, operation "Succeeded"
Healthy
Wall time: 1min 2s


# 6. Test du Web Service déployé


In [56]:
%%time
import json

test_sample = json.dumps({'data': [
    [1,2,3,4,5,6,7,8,9,10]
]})
test_sample = bytes(test_sample,encoding = 'utf8')

print("- Entrées fournies au web service :")
print(test_sample)
print()
prediction = aks_service.run(input_data = test_sample)
print("- Prédictions obtenues par le web service :")
print(prediction)
print()

- Entrées fournies au web service :
b'{"data": [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]}'

- Prédictions obtenues par le web service :
[5215.1981315798685]

Wall time: 251 ms


# 7. Nettoyage
Supression AKS, image et modèle

In [54]:
#aks_service.delete()
#image.delete()
#model.delete()